In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
dataset_cancer = pd.read_csv(r"../datasets/enfermedades/casos_cancer_pulmon_fym_NORMALIZADO.csv",sep=';')
dataset_co2 = pd.read_csv(r"../datasets/energyco2_normalizado.csv",sep=',')

In [3]:
df_cancer = pd.DataFrame(dataset_cancer)
df_co2 = pd.DataFrame(dataset_co2)
col = 'Unnamed: 0'

In [4]:
df_cancer.drop(columns=col,inplace=True)
df_co2.drop(columns=col,inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [5]:
df_co2.rename(columns={'co2_emision_co2':'co2'},inplace=True)

In [6]:
df_cancer = df_cancer.merge(df_co2,on=['pais','anio'])

In [7]:
df_cancer = df_cancer[df_cancer['energia_tipo']=='all_energy_types']

In [8]:
paises = ['United States', 'China', 'Russia', 'Saudi Arabia', 'Canada']
cols = ['pais','anio','casos F','Casos M','energia_produccion','co2']

In [9]:
df2 = df_cancer[cols]

In [10]:
df_cancerTOP = df2[df_cancer['pais'].isin(paises)]

In [11]:
df3 = df_cancerTOP.groupby(by='pais').sum().reset_index()

In [12]:
df3['casos'] = df3['casos F'] + df3['Casos M']

In [13]:
df3.query("pais=='Saudi Arabia'")

,pais,anio,casos F,Casos M,energia_produccion,co2,casos
3,Saudi Arabia,60105,6494,20066,699.96,10849.67,26560


In [14]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df3['pais'],y=df3['co2'],name='CO2'))
fig.add_trace(go.Bar(x=df3['pais'],y=df3['energia_produccion'],base='stack',name='Produccion',opacity=0.6),secondary_y=False)
fig.add_trace(go.Scatter(x=df3['pais'],y=df3['casos'],name='Casos',opacity=0.5),secondary_y=True)
fig.update_layout(go.Layout(
    barmode='overlay'
))
fig.update_layout(title='Consumo vs produccion en top paises productores energia',title_x = 0.4)
fig.show()

In [15]:
df3[['casos','co2']].corr()

,casos,co2
casos,1.000000,0.893207
co2,0.893207,1.000000


KPI 2030

In [16]:
df = df_co2[df_co2['energia_tipo']=='all_energy_types']
df = df[['pais','anio','energia_cons','co2','pbi']]

In [17]:
df = df.sort_values(by=['pais','anio']).reset_index(drop=True)

In [19]:
df['pct_chg_co2'] = df['co2'].pct_change(1)*100
df['pct_chg_cons'] = df['energia_cons'].pct_change(1)*100
df = df[df['anio']>=2010]

In [40]:
df.pais.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Angola',
       'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia',
       'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo-Brazzaville', 'Congo-Kinshasa', 'Cook Islands',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Côte d’Ivoire', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Falkland Islands', 'Faroe Islands', 'Fiji', 

In [41]:
df.to_csv('../src/app/utils/top5.csv',index=False)

In [20]:
paises =['United States', 'China', 'Russia', 'Saudi Arabia', 'Canada']

In [21]:
df_top = df[df['pais'].isin(paises)]

In [22]:
fig = px.line(df_top,x='anio',y='pct_chg_co2',color='pais')
fig.update_layout(title="Porcentaje de cambio en emisiones de CO2, TOP5 paises emisores")
fig.show()

In [23]:
df_kpi_2030 = df.groupby(by=['pais','anio']).sum()
df_kpi_2030 = df_kpi_2030.reset_index()
kpi = df_kpi_2030.loc[df_kpi_2030['anio']==2019,['pais','pct_chg_co2']]
kpi.sort_values(by='pct_chg_co2').head(15)

,pais,pct_chg_co2
1399,Mozambique,-27.467412
1229,Macau,-18.292683
759,Gabon,-15.681234
189,Belize,-14.705882
2059,Tonga,-11.764706
599,Equatorial Guinea,-11.553030
1819,Seychelles,-11.538462
19,Albania,-10.044643
1239,Madagascar,-8.378378
2289,Yemen,-8.296460


In [24]:
top_emisores=['United States', 'China', 'Russia', 'Saudi Arabia', 'Canada']

In [25]:
anios = [2010,2019]
df19 = df[df['anio'].isin(anios)][['pais','pbi','anio','co2']]
df19['pct_chg_co2'] = df19['co2'].pct_change(1)*100
df19 = df19[df19['anio']==2019]
df19['cumplimiento']=(df19['pct_chg_co2']+45)/-45*100

In [26]:
df19

,pais,pbi,anio,co2,pct_chg_co2,cumplimiento
39,Afghanistan,79.45,2019,9.39,50.722311,-212.716248
79,Albania,38.27,2019,4.03,33.443709,-174.319352
119,Algeria,509.07,2019,145.99,46.900785,-204.223966
159,American Samoa,NaN,2019,0.36,-40.000000,-11.111111
199,Angola,193.83,2019,17.64,42.834008,-195.186685
...,...,...,...,...,...,...
9079,Western Sahara,NaN,2019,0.26,-13.333333,-70.370370
9119,World,127690.25,2019,35584.93,16.207718,-136.017152
9159,Yemen,53.08,2019,8.29,-60.429594,34.287987
9199,Zambia,247.18,2019,7.10,200.847458,-546.327684


In [27]:
df19.query("pais != 'World'").sort_values(by='co2',ascending=False).head(5)

,pais,pbi,anio,co2,pct_chg_co2,cumplimiento
1639,China,23128.34,2019,10608.60,29.307886,-165.128636
8799,United States,19925.43,2019,5277.19,-2.340451,-94.798998
3959,India,9310.29,2019,2308.33,49.256733,-209.459406
6799,Russia,3770.42,2019,1798.24,19.437563,-143.194584
4279,Japan,5359.05,2019,1140.95,-0.343267,-99.237184


In [28]:
top_emision = df19.query("pais != 'World'").sort_values(by='co2',ascending=False).head(5)
top_reduccion = df19.sort_values(by='pct_chg_co2').head(5)
df_pbi = pd.concat([top_emision,top_reduccion])
df_pbi.dropna(subset='pbi',inplace=True)

In [29]:
df_pbi['cumplimiento'] = df_pbi['cumplimiento'].round(2)

In [30]:
df_pbi = df_pbi.sort_values(by='pbi',ascending=False)

In [31]:
title = f"Relacion PBI - cumplimiento de -45% CO2 entre TOP5 emision y TOP5 paises cumplimiento de meta (2019)"

In [32]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_pbi['pais'],y=df_pbi['pbi'],name='PBI'))
fig.add_trace(go.Scatter(x=df_pbi['pais'],y=df_pbi['cumplimiento'],name='Cumplimiento',mode='markers', marker_color=df_pbi['cumplimiento'],),secondary_y=True)
fig.update_layout(title=title)
fig.show()

In [33]:
data = pd.read_csv(r"C:\Users\ferch\Desktop\Henry\Proyecto Grupal\CO2-Emissions\datasets\NORMALIZADO_central_electrica.csv",sep=';')
df_pp = pd.DataFrame(data)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ferch\\Desktop\\Henry\\Proyecto Grupal\\CO2-Emissions\\datasets\\NORMALIZADO_central_electrica.csv'

In [ ]:
df_pp.drop(columns=col,inplace=True)

In [ ]:
df_pp

,codigo_iso,pais,nombre,capacidad_MW,latitud,longitud,energia_primaria,otra_energia1,año_apertura,año_capacidad_reportada,anio,GWh_x_anio
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.3220,65.1190,Hydro,NaN,NaN,2017.0,2013,NaN
1,AFG,Afghanistan,Kandahar DOG,10.0,31.6700,65.7950,Solar,NaN,NaN,NaN,2013,NaN
2,AFG,Afghanistan,Kandahar JOL,10.0,31.6230,65.7920,Solar,NaN,NaN,NaN,2013,NaN
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.5560,69.4787,Hydro,NaN,NaN,2017.0,2013,NaN
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.6410,69.7170,Hydro,NaN,NaN,2017.0,2013,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
244547,ZMB,Zambia,Ndola,50.0,-12.9667,28.6333,Oil,NaN,NaN,NaN,2019,NaN
244548,ZMB,Zambia,Nkana,20.0,-12.8167,28.2000,Oil,NaN,NaN,NaN,2019,NaN
244549,ZMB,Zambia,Victoria Falls,108.0,-17.9167,25.8500,Hydro,NaN,NaN,NaN,2019,NaN
244550,ZWE,Zimbabwe,Hwange Coal Power Plant Zimbabwe,920.0,-18.3835,26.4700,Coal,NaN,NaN,2017.0,2019,NaN


In [ ]:
contaminantes = ['Gas','Oil','Coal','Petcoke','Waste','Biomass']

In [ ]:
df_pp =df_pp[df_pp['energia_primaria'].isin(contaminantes)]

In [ ]:
df_pp['text'] = df_pp['nombre'] + ',' + df_pp['pais'] + ', ' + df_pp['energia_primaria'] + '' + 'Capacidad: ' + df_pp['capacidad_MW'].astype(str)

fig = go.Figure(data=go.Scattergeo(
        lon = df_pp['longitud'],
        lat = df_pp['latitud'],
        text = df_pp['text'],
        mode = 'markers',
        marker_color = df_pp['capacidad_MW'], 
        opacity=0.8
        ))

fig.update_layout(
        title = 'Plantas productoras de energia contaminantes',
        title_x = 0.4,
        geo_scope='world',
    )
fig.show()

In [ ]:
df_co2.energia_tipo.unique()

array(['all_energy_types', 'coal', 'natural_gas',
       'petroleum_n_other_liquids', 'nuclear', 'renewables_n_other'],
      dtype=object)

In [ ]:
energias_limpias = ['nuclear', 'renewables_n_other',]
df_er = df_co2[df_co2['energia_tipo'].isin(energias_limpias)][['pais','anio','energia_tipo','energia_produccion','co2']]

In [ ]:
dd = df_er.groupby(by=['pais','anio']).sum().reset_index()

In [ ]:
df_ae = df_co2[['pais','anio','energia_tipo','energia_produccion','co2']]
df_ae = df_ae[df_ae['energia_tipo']=='all_energy_types']
df_ae.rename(columns={'energia_produccion':'prod','co2':'co2_T'},inplace=True)
df_ae.head()

,pais,anio,energia_tipo,prod,co2_T
0,World,1980,all_energy_types,296.34,4946.63
6,Afghanistan,1980,all_energy_types,0.07,NaN
12,Albania,1980,all_energy_types,0.16,NaN
18,Algeria,1980,all_energy_types,2.80,NaN
24,American Samoa,1980,all_energy_types,0.00,NaN


In [ ]:
dd = dd.merge(df_ae,on=['pais','anio'])

In [ ]:
dd['porcentaje_renovables'] = dd['energia_produccion']/dd['prod']*100
dd.drop(columns=['co2','energia_tipo'],inplace=True)
dd.rename(columns={'co2_T':'CO2'},inplace=True)

In [ ]:
dd = dd.round(2)
dd.head()

,pais,anio,energia_produccion,prod,CO2,porcentaje_renovables
0,Afghanistan,1980,0.01,0.07,NaN,14.29
1,Afghanistan,1981,0.01,0.09,1.33,11.11
2,Afghanistan,1982,0.01,0.10,1.51,10.00
3,Afghanistan,1983,0.01,0.10,1.66,10.00
4,Afghanistan,1984,0.01,0.10,2.06,10.00


In [ ]:
paises

['United States', 'China', 'Russia', 'Saudi Arabia', 'Canada']

In [ ]:
paises.append('Argentina')

In [ ]:
df_plot = dd[dd['anio'] >=2010 ]

In [ ]:
fig = px.line(df_plot[df_plot['pais'].isin(paises)],x='anio',y='porcentaje_renovables',color='pais')
fig.update_layout(
    title="Evolucion de la participación de energías limpias en total, TOP5 productores + Arg",
    xaxis_title="Año",
    yaxis_title="Porcentaje de energías renovables sobre total",
)
fig.show()